In [1]:
#March 10, 2020
#Notebook used to download new Twitter Data
#Imports Twitter Library And Set Authorization Tokens From Twitter App Page
#Creates authorization object that can read from Twitter
import twitter
consumer_key = "I3MtCPEfecYhtXiTZ5KYVWphk"
consumer_secret = "1J8jSkBBrmUE7EJKofdnKuKMsxWrF3ajPmcTGakMuNGyzKWpKl"
access_token = "1118431530-7PJHNpHjCZA3uKY0etuszV5szKz6QVdNsXjjVhX"
access_token_secret = "8HdBjlwSQ6FRRO37xGw23UoQBBIROsA6KgTG8vjBCzdwf"
authorization = twitter.OAuth(access_token, access_token_secret, consumer_key, consumer_secret)
t = twitter.Twitter(auth=authorization, retry=True)

In [2]:
#Set up filenames 
import os 
data_folder = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop", "Data Mining", "Chapter7HW", "Data", "twitter")
output_filename = os.path.join(data_folder, "python_tweets.json")

In [3]:
#Creates a list of users by searching for tweets and look for those that mention 'python'
#Creates two lists for storing the text of the tweet and the corresponding users
#Creates a dictionary mapping for the user ID's
original_users = [] 
tweets = []
user_ids = {}

In [4]:
#Search for the word 'python' and iterate over those search results
#We look for the Tweets that have text as the last example
search_results = t.search.tweets(q="python", count=100)['statuses']
for tweet in search_results:
    if 'text' in tweet:
        original_users.append(tweet['user']['screen_name']) 
        user_ids[tweet['user']['screen_name']] = tweet['user']['id']
        tweets.append(tweet['text'])

In [7]:
#Import the library and create an output filename for the model (Extra)
#Store in Models directory
#from sklearn.externals import joblib
#output_filename = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop", "Data Mining", "Chapter7HW", "Models", "twitter", "python_context.pkl")

In [8]:
#Use dump function to pass model by itself and output_filename (Extra)
#joblib.dump(model, output_filename)

In [9]:
#Set the filename of the model
model_filename = os.path.join(os.path.expanduser("~"), "OneDrive", "Desktop", "Data Mining", "Chapter7HW", "Models", "twitter", "python_context.pkl")

In [10]:
#Recreate the BagofWords class with dependencies
import spacy
from sklearn.base import TransformerMixin

# Create a spaCy parser
nlp = spacy.load('en')


class BagOfWords(TransformerMixin):
    def fit(self, X, y=None):
        return self
 
    def transform(self, X):
        results = []
        for document in X:
            row = {}
            for word in list(nlp(document, tag=False, parse=False, entity=False)):
                if len(word.text.strip()): # Ignore words that are just whitespace
                    row[word.text] = True
                    results.append(row)
        return results

In [11]:
#Imports the joblib library to facilitate the saving and loading models 
#Call the load function
#from sklearn.externals 
import joblib
context_classifier = joblib.load(model_filename)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\JRuiz\\OneDrive\\Desktop\\Data Mining\\Chapter7HW\\Models\\twitter\\python_context.pkl'

In [ ]:
#Context_Classifier is an instance of a Pipeline
#Calling this function gives us a prediction to see if the tweets are relevant to the python programming language
y_pred = context_classifier.predict(tweets)

In [ ]:
#The ith item in y_pred will be 1 if th ith tweet is predicted to be related to 'python' as a programming language or else it is 0
#This helps us get the tweets that are relevant and the relevant users
relevant_tweets = [tweets[i] for i in range(len(tweets)) if y_pred[i] == 1]
relevant_users = [original_users[i] for i in range(len(tweets)) if y_pred[i] == 1]

In [12]:
#Helps us get follower information using Twitter's pagination function
#Twitter returns information call using a cursor which will be an integer, or 0 if there is no more information
#Loops through the function while the cursor is not equal to 0
#Request user's followers and add them to the list
#Use try block/exception handling to handle errors if needed
#Use followers ID to store in the ID key of the results dictionary, then update the cursor
#If > 10000 friends break the loop
import time

def get_friends(t, user_id):
    friends = []
    cursor = -1
    while cursor != 0: 
        try:
            results = t.friends.ids(user_id= user_id, cursor=cursor, count=5000)
            friends.extend([friend for friend in results['ids']])
            cursor = results['next_cursor'] 
            if True or len(friends) >= 10000:
                break
        except TypeError as e:
            if results is None:
                print("You probably reached your API limit, waiting for 5 minutes")
                sys.stdout.flush() 
                time.sleep(5*60) # 5 minute wait 
            else: 
                # Some other error happened, so raise the error as normal
                raise e
        except twitter.TwitterHTTPError as e:
            print(e)
            break
        finally:
            # Break regardless -- this stops us going over our API limit
            time.sleep(60)
    return friends

In [13]:
#Build network of users, creating a link if two user follow each other
#Create data structure that can segment list of users into groups, and recommend users to each other 
#Get friends and store them in a dictionary, after obtaining user ID from dictionary
friends = {} 
for screen_name in relevant_users:
    user_id = user_ids[screen_name]
    friends[user_id] = get_friends(t, user_id)

NameError: name 'relevant_users' is not defined

In [14]:
#Remove any users with no friends as they do not pertain to the dataset in question
friends = {user_id:friends[user_id] 
           for user_id in friends
           if len(friends[user_id]) > 0}

In [15]:
#Get friends of existing users by starting with stronger connections
#Maintain a count of all the times the user is in the friends list
#Iterate over friends lists and count each time a friend occurance happens
from collections import defaultdict
def count_friends(friends): 
    friend_count = defaultdict(int)
    for friend_list in friends.values(): 
        for friend in friend_list:
            friend_count[friend] += 1 
    return friend_count

In [16]:
#Get the most connected person in the sample
friend_count = count_friends(friends)
from operator import itemgetter
best_friends = sorted(friend_count, key=friend_count.get, reverse=True)

In [17]:
#Outputs best friend count 
best_friends

[]

In [18]:
#Imports sys library
import sys

In [12]:
#Set up a loop that runs until the friends of 150 users are obtained, iterate over best friends based on number of people until finding a new unchecked user
while len(friends) < 150:
    for user_id in best_friends:
        if user_id in friends:
            continue
        print(user_id)
        sys.stdout.flush()
        friends[user_id] = get_friends(t, user_id) 
        for friend in friends[user_id]: 
            friend_count[friend] += 1
        best_friends = sorted(friend_count.items(), key=itemgetter(1), reverse=True)
        break

NameError: name 'friends' is not defined

In [13]:
#Gets length of amount of users
len(friends)

NameError: name 'friends' is not defined

In [14]:
#Save friend's dictionary to a file with json
import json
friends_filename = os.path.join(data_folder, "python_friends.json")

In [15]:
#Saves file
with open(friends_filename, 'w') as outf: 
    json.dump(friends, outf)

NameError: name 'friends' is not defined

In [16]:
#Load the file using json.load function
with open(friends_filename) as inf:
    friends = json.load(inf)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#Display length and type
len(friends), type(friends)

In [17]:
#Import networkx library and set a Directed Graph
import networkx as nx 
G = nx.DiGraph()

In [18]:
#Visualize the key users and add them to graph nodes
main_users = friends.keys() 
G.add_nodes_from(main_users)

NameError: name 'friends' is not defined

In [ ]:
#Create edges in graph frmo one user to the other user if the second user is a friend of the first
#Iterate through all friends of given user if the friend is a main user and add the edge if so
for user_id in friends:
    for friend in friends[user_id]:
        if str(friend) in main_users: 
            G.add_edge(user_id, friend) 

In [ ]:
#Test draw
nx.draw?

In [ ]:
#Use the draw function from NetworkX which utilizes matplotlib
%matplotlib inline 
nx.draw(G)

In [ ]:
#Improve the graph quality by using pyplot to handle the creation of the figure, used by NetworkX to do graph drawing
#Call draw function to increase size of the larger figure that is created
from matplotlib import pyplot as plt
plt.figure(3,figsize=(20,20))
nx.draw(G, alpha=0.1, edge_color='b')

In [ ]:
#Begin to computer Jaccard similarity/coefficient, divide the intersection of two sets of followers by the union of those two, using set operators
#Convert lists to set by using
friends = {user: set(friends[user]) for user in friends}

In [ ]:
#Create function that computes similarity between two sets of lists of friends
#Used 1e-6 to ensure there is never a division by 0 error in the case where a user has no friends
def compute_similarity(friends1, friends2):
    return len(friends1 & friends2) / (len(friends1 | friends2) + 1e-6)

In [ ]:
#Create the weighted graph of similarity between users 
def create_graph(followers, threshold=0): 
    G = nx.Graph()
    for user1 in friends.keys(): 
        for user2 in friends.keys(): 
            if user1 == user2:
                continue
            weight = compute_similarity(friends[user1], friends[user2])
            if weight >= threshold:
                G.add_node(user1) 
                G.add_node(user2)
                G.add_edge(user1, user2, weight=weight)
    return G

In [ ]:
#Create the graph by calling the function, starting with no threshold
G = create_graph(friends)

In [ ]:
#Strongly connected graph, resize the image
#Use spring_layout for visualization
#Position the nodes using pos
#Draw the edges and iterate over edges to get the weights
#Drae the edges fully
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos)
edgewidth = [ d['weight'] for (u,v,d) in G.edges(data=True)]
nx.draw_networkx_edges(G, pos, width=edgewidth)

In [ ]:
#Create a new graph with a threshold of 0.1 to get edges with at least that weight, 10% of followers between two users
G = create_graph(friends, 0.1)

In [ ]:
#Defines subgraphs before use
def connected_component_subgraphs(G):
    for c in nx.connected_components(G):
        yield G.subgraph(c)

In [ ]:
#use Network X to get connected components
sub_graphs = nx.connected_component_subgraphs(G)

In [ ]:
#Iterate over groups and print out sizes of graph
for i, sub_graph in enumerate(sub_graphs):
    n_nodes = len(sub_graph.nodes()) 
    print("Subgraph {0} has {1} nodes".format(i, n_nodes))

In [ ]:
#Alter the threshold to alter connected components, makes for fewer edges connecting nodes and smaller connected components with more quantity
G = create_graph(friends, 0.25) 
sub_graphs = nx.connected_component_subgraphs(G) 
for i, sub_graph in enumerate(sub_graphs): 
    n_nodes = len(sub_graph.nodes()) 
    print("Subgraph {0} has {1} nodes".format(i, n_nodes))

In [ ]:
#Gets connected components and count of connected components
sub_graphs = nx.connected_component_subgraphs(G) 
n_subgraphs = nx.number_connected_components(G)

In [ ]:
#Display count of connected components
n_subgraphs

In [ ]:
#Create new pyplot figure and show all connected components
#Iterate over each connected component and add subplot for each one
#Parameters include rows, columns and index
sub_graphs = nx.connected_component_subgraphs(G) 
n_subgraphs = nx.number_connected_components(G)

fig = plt.figure(figsize=(20, (n_subgraphs * 3)))
for i, sub_graph in enumerate(sub_graphs):
    
    ax = fig.add_subplot(int(n_subgraphs / 3)+1, 3, i+1)
    ax.get_xaxis().set_visible(False) 
    ax.get_yaxis().set_visible(False)
    nx.draw(sub_graph, ax=ax)

In [ ]:
#Display when complete
print("Done")

In [ ]:
#Creates a function to take in threshold values and friends list, compute Silhouette Coefficient by using a matrix from the graph 
#Uses NetworkX's to_scipy_sparse_matrix
import numpy as np
from sklearn.metrics import silhouette_score

def compute_silhouette(threshold, friends):
    G = create_graph(friends, threshold=threshold) 
    if len(G.nodes()) < 2:
        return -99
    sub_graphs = nx.connected_component_subgraphs(G)

    if not (2 <= nx.number_connected_components(G) < len(G.nodes()) - 1): 
        return -99

    label_dict = {}
    for i, sub_graph in enumerate(sub_graphs): 
        for node in sub_graph.nodes(): 
            label_dict[node] = i

    labels = np.array([label_dict[node] for node in G.nodes()])
    X = nx.to_scipy_sparse_matrix(G).todense()
    X = 1 - X
    return silhouette_score(X, labels, metric='precomputed')

In [ ]:
#Creates the inverse function to minimize as we are interested in showing the improvement of lower scores
#Creates new function to pass the same arguments and keywords but negate the value before returning
def inverted_silhouette(threshold, friends):
    return -compute_silhouette(threshold, friends)

In [ ]:
#Call minimize function on the inverted function
from scipy.optimize import minimize
result = minimize(inverted_silhouette, 0.1, args=(friends,))

In [ ]:
#Display full result
result